In [4]:
import sys
sys.path.append('../implementation/')
import ast
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.special as sp
from tqdm import tqdm
import time
from healey_adaboost_naive_bayes import AdaBoostNB
import warnings
from util import flatten_list
warnings.filterwarnings('ignore')

In [5]:
# Loading the STL Crimes underlying data and user interaction data
data_path = '../data/boardrooms/boardrooms_data.csv'
ui_data_path = '../data/boardrooms/boardrooms_combined_interactions.csv'
output_file_path = '../output/boardrooms/boardrooms_ada_nb.pkl'

underlying_data = pd.read_csv(data_path)
underlying_data['industry_code'] = pd.factorize(underlying_data['industry'])[0]
interaction_data = pd.read_csv(ui_data_path)
interaction_data['interaction_session'] = interaction_data.apply(lambda row: ast.literal_eval(row.interaction_session), axis=1)

ks = [1, 5, 10, 20, 50, 100]
d_attributes = ['industry_code']
c_attributes = ['mktcap', 'unrelated', 'female', 'age', 'tenure', 'medianpay']

underlying_data = underlying_data[d_attributes + flatten_list(c_attributes)].copy()

In [6]:
nb_results = pd.DataFrame()

for participant_index, row in interaction_data.iterrows():
    print(f'Processing user {row.user} task {row.task}')
    results = {'participant_id': row.user, 'task': row.task}
    model = AdaBoostNB(underlying_data, c_attributes, d_attributes)
    predicted = pd.DataFrame()
    rank_predicted = []
    for i in tqdm(range(len(interaction_data.iloc[participant_index].interaction_session))):
        interaction = interaction_data.iloc[participant_index].interaction_session[i]
        model.update(interaction)

        if i < len(interaction_data.iloc[participant_index].interaction_session) - 1:
            probability_of_next_point = model.predict()
            next_point = interaction_data.iloc[participant_index].interaction_session[i+1]
            predicted_next_dict = {}
            for k in ks:
                predicted_next_dict[k] = (next_point in probability_of_next_point.nlargest(k).index.values)
            predicted = predicted.append(predicted_next_dict, ignore_index=True)
            sorted_prob = probability_of_next_point.sort_values(ascending=False)
            rank, = np.where(sorted_prob.index.values == next_point)
            rank_predicted.append(rank[0] + 1)
    ncp = predicted.sum()/len(predicted)
    for col in ncp.index:
        results[f'ncp-{col}'] = ncp[col]
    results['rank'] = rank_predicted  
    nb_results = nb_results.append(results, ignore_index=True)
    
nb_results.to_pickle(output_file_path)

Processing user 1 task 1


100%|██████████| 20/20 [00:01<00:00, 11.73it/s]


Processing user 4 task 1


100%|██████████| 14/14 [00:01<00:00, 10.68it/s]


Processing user 12 task 1


...

100%|██████████| 15/15 [00:01<00:00, 10.87it/s]


Processing user 183 task 1


100%|██████████| 20/20 [00:01<00:00, 11.35it/s]


Processing user 185 task 1


100%|██████████| 14/14 [00:01<00:00, 12.38it/s]
